In [18]:
from IPython.display import HTML
import cPickle as pickle
import pandas as pd
import numpy as np

from data_prep import parse_tickets, parse_payouts, get_data
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
data = pd.read_json('data.json')

## Constructing data for model

In [3]:
model_data = data[['acct_type', 'ticket_types', 'previous_payouts', 'user_created', 'user_age', 'event_start', 'event_created', 'approx_payout_date', 'name_length']]

model_data['avg_ticket_sold_cnt'] = model_data['ticket_types'].map(lambda x: parse_tickets(x)[3])
model_data['event_age_at_start(days)'] = (model_data['event_start'] - model_data['event_created'])/86400.0
model_data['total_payout_hist'] = model_data['previous_payouts'].map(lambda x: parse_payouts(x)[2])
model_data['payout'] = model_data['ticket_types'].map(lambda x: parse_tickets(x)[4])
model_data['num_payouts_hist'] = model_data['previous_payouts'].map(lambda x: parse_payouts(x)[0])
model_data['avg_ticket_cost'] = model_data['ticket_types'].map(lambda x: parse_tickets(x)[1])

model_data = model_data.drop(['ticket_types', 'previous_payouts', 'event_start', 'event_created'], axis=1)

/home/michael/anaconda2/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/michael/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/michael/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/michael/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [4]:
label_map = {'fraudster': 1, 'fraudster_event': 1, 'locked': 1, 'premium': 1, 'spammer_limited': 1, 'spammer_noinvite': 1, 'spammer_warn': 1, \
             'tos_warn': 1, 'premium': 0}

model_data['label'] = model_data['acct_type'].map(label_map)

In [5]:
model_data.groupby('label').agg('count')

,acct_type,user_created,user_age,approx_payout_date,name_length,avg_ticket_sold_cnt,event_age_at_start(days),total_payout_hist,payout,num_payouts_hist,avg_ticket_cost
label,,,,,,,,,,,
0.0,12373,12373,12373,12373,12373,12370,12373,12373,12373,12373,12370
1.0,1879,1879,1879,1879,1879,1795,1879,1879,1879,1879,1795


In [6]:
model_data = model_data.dropna()
y = model_data['label'].values
X = model_data.drop(['label', 'acct_type'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y)

## Cost Functions

In [7]:
def fraud_cost(estimator, X, y):
    compare = pd.DataFrame(data=[y, estimator.predict(X), -X[:, 8]], index=['actual', 'pred', 'cost'])
    compare = compare.T
    compare['confusion'] = zip(compare['actual'], compare['pred'] == compare['actual'])
    compare['count'] = 1
    conf_map = {(0.0, False): 'FP', (0.0, True): 'TN', (1.0, False): 'FN', (1.0, True): 'TP'}
    compare['confusion'] = compare['confusion'].map(conf_map)
    compare.loc[compare['confusion'] == 'TP', 'cost'] = -500
    compare.loc[compare['confusion'] == 'FP', 'cost'] = -500
    compare.loc[compare['confusion'] == 'TN', 'cost'] = 0
    total_cost = compare.cost.agg('sum')
    return compare['cost'].values

In [8]:
def my_cost(estimator, X, y):
    compare = pd.DataFrame(data=[y, estimator.predict(X), -X[:, 8]], index=['actual', 'pred', 'cost'])
    compare = compare.T
    compare['confusion'] = zip(compare['actual'], compare['pred'] == compare['actual'])
    compare['count'] = 1
    conf_map = {(0.0, False): 'FP', (0.0, True): 'TN', (1.0, False): 'FN', (1.0, True): 'TP'}
    compare['confusion'] = compare['confusion'].map(conf_map)
    compare.loc[compare['confusion'] == 'TP', 'cost'] = -500
    compare.loc[compare['confusion'] == 'FP', 'cost'] = -500
    compare.loc[compare['confusion'] == 'TN', 'cost'] = 0
    total_cost = compare.cost.agg('sum')
    conf_matrix = compare[['confusion', 'count']].groupby('confusion').agg('count')
    return total_cost, conf_matrix

## Forest Model

In [9]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
confusion_matrix(y_test, forest.predict(X_test))

array([[3076,    7],
       [ 156,  303]])

In [10]:
my_cost(forest, X_test, y_test)

(-163685.0,            count
 confusion       
 FN           156
 FP             7
 TN          3076
 TP           303)

In [11]:
fraud_cost(forest, X_test, y_test)

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

## Adaboost Model Gridsearch

In [12]:
ada = AdaBoostClassifier()
ada_param = {'n_estimators': [75, 80, 85, 90], 'learning_rate': [0.65, 0.7], 'random_state': [1]}
grid = GridSearchCV(ada, ada_param)
grid.fit(X_train, y_train)
confusion_matrix(y_test, grid.predict(X_test))

array([[3072,   11],
       [ 174,  285]])

In [13]:
grid.best_params_

{'learning_rate': 0.7, 'n_estimators': 90, 'random_state': 1}

In [14]:
my_cost(grid.best_estimator_, X_test, y_test)

(-153970.0,            count
 confusion       
 FN           174
 FP            11
 TN          3072
 TP           285)

## Model pickling/unpickling

In [16]:
with open('model.pkl', 'w') as f:## Parsing ticket data
    pickle.dump(grid.best_estimator_, f)

In [32]:
with open('model.pkl') as f2:
    model = pickle.load(f2)

model.feature_importances_

array([ 0.16666667,  0.03333333,  0.08888889,  0.04444444,  0.07777778,
        0.13333333,  0.15555556,  0.04444444,  0.21111111,  0.04444444])